https://dmnfarrell.github.io/bioinformatics/abm-mesa-python

https://royalsociety.org/-/media/policy/projects/set-c/set-covid-19-R-estimates.pdf

# Mesa

In [15]:
import time
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

Primero se define la clase para el modelo de infección cuyos parámetros son:

- Número de agentes
- Probabilidad de transmisión
- Tasa de mortalidad
- Tiempo de recuperación en días
- Desviación estándar de la recuperación 

In [16]:
class InfectionModel(Model):
    """A model for infection spread."""

    def __init__(self, N=10, width=10, height=10, ptrans=0.5,
                 death_rate=0.02, incubation=7, incubation_sd=3, recovery_days=21,
                 recovery_sd=7):

        self.num_agents = N
        self.incubation_period = incubation
        self.incubation_sd = incubation_sd
        self.recovery_days = recovery_days
        self.recovery_sd = recovery_sd
        self.ptrans = ptrans
        self.death_rate = death_rate
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True
        self.dead_agents = []
        # Create agents
        for i in range(self.num_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            # Make some agents infected at start
            infected = np.random.choice([0,1], p=[0.98,0.02])
            if infected == 1:
                a.state = State.INFECTED
                a.recovery_time = self.get_recovery_time()

        self.datacollector = DataCollector(          
            agent_reporters={"State": "state"})

    def get_incubation_time(self):
        return int(self.random.normalvariate(self.incubation_days,self.incubation_sd))
        
    def get_recovery_time(self):
        return int(self.random.normalvariate(self.recovery_days,self.recovery_sd))

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

$R_0$ represents the basic reproduction number,
which is the number of secondary infections generated from
an initial case at the beginning of an epidemic, in an entirely
susceptible population

$R_0 = 2.5$ for COVID-19

$p = 1 - \dfrac{1}{R_0} = 0.6$ the critical proportion of the population that must be immune if
transmission is to be halted

$R_0 = \dfrac{\beta}{\sigma}N$

$N$ population size

$\beta$ transmission parameter

$\dfrac{1}{\sigma}$ average duration of infectiousness

In [17]:
class State(Model):
    SUSCEPTIBLE = 0
    INFECTED = 1 #Infected and infectious
    REMOVED = 2
    EXPOSED = 3 #Infected but not infectious

class MyAgent(Agent):
    """ An agent in an epidemic model."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.age = self.random.normalvariate(20,40)        
        self.state = State.SUSCEPTIBLE  
        self.infection_time = 20

    def move(self):
        """Move the agent"""
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def contact(self):
        """Find close contacts and infect"""
        cellmates = self.model.grid.get_cell_list_contents([self.pos])       
        if len(cellmates) > 1:
            for other in cellmates:
                if self.random.random() > model.ptrans:
                    continue
                if self.state is State.INFECTED and other.state is State.SUSCEPTIBLE:                    
                    other.state = State.INFECTED
                    other.infection_time = self.model.schedule.time
                    other.recovery_time = model.get_recovery_time()

    def status(self):
        """Check infection status"""
        if self.state == State.INFECTED:     
            drate = self.model.death_rate
            alive = np.random.choice([0,1], p=[drate, 1-drate])
            if alive == 0: #if it's alive
                self.model.schedule.remove(self)            
            t = self.model.schedule.time - self.infection_time
            if t >= self.recovery_time:
                self.state = State.REMOVED

    def step(self):
        self.move()
        self.contact()
        self.status()

In [18]:
pop = 100
steps = 100
model = InfectionModel(pop, 20, 20, ptrans=0.5)
for i in range(steps):
    model.step()
agent_state = model.datacollector.get_agent_vars_dataframe()

In [19]:
agent_state

State
Step AgentID       
0    0            0
     1            0
     2            0
     3            0
     4            0
...             ...
99   95           2
     96           2
     97           2
     98           2
     99           2

[9160 rows x 1 columns]

In [20]:
pop = 1000
steps = 10
model = InfectionModel(pop, 20, 20, ptrans=0.5)
for i in range(steps):
    model.step()
agent_state = model.datacollector.get_agent_vars_dataframe()

In [21]:
def get_column_data(model):
    """pivot the model dataframe to get states count at each step"""
    agent_state = model.datacollector.get_agent_vars_dataframe()
    X = pd.pivot_table(agent_state.reset_index(),index='Step',columns='State',aggfunc=np.size,fill_value=0)    
    labels = ['Susceptible','Infected','Removed']
    X.columns = labels[:len(X.columns)]
    return X

In [22]:
get_column_data(model)

,Susceptible,Infected,Removed
Step,,,
0,983,17,0
1,954,45,0
2,890,108,0
3,759,236,1
4,569,423,1
5,367,616,2
6,199,772,2
7,111,838,9
8,47,879,12


# Bokeh

In [23]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10
from bokeh.models.glyphs import Line
from bokeh.layouts import gridplot
from bokeh.models import Legend, LegendItem
from bokeh.models.mappers import LinearColorMapper
import panel as pn

In [24]:
def plot_states_bokeh(model,title=''):
    """Plot cases per country"""

    X = get_column_data(model)
    X = X.reset_index()
    source = ColumnDataSource(X)
    i=0
    colors = Category10[3]
    items=[]
    p = figure( plot_width=600, plot_height=400, tools=[], title=title, x_range=(0,80) )
    for c in X.columns[1:]:
        line = Line(x='Step',y=c, line_color=colors[i],line_width=3,line_alpha=.8,name=c)
        glyph = p.add_glyph(source, line)
        i+=1
        items.append((c,[glyph]))

    p.xaxis.axis_label = 'Step'
    p.add_layout(Legend(location='center_right',   
                items=items))
    p.background_fill_color = "#e1e1ea"
    p.background_fill_alpha = 0.5
    p.legend.label_text_font_size = "10pt"
    p.title.text_font_size = "15pt"
    p.toolbar.logo = None
    p.sizing_mode = 'scale_height'    
    return p

In [25]:
def grid_values(model):
    """Get grid cell states"""

    agent_counts = np.zeros((model.grid.width, model.grid.height))
    w=model.grid.width
    df=pd.DataFrame(agent_counts)
    for cell in model.grid.coord_iter():
        agents, x, y = cell
        c=None
        for a in agents:
            c = a.state
        df.iloc[x,y] = c
    return df

def plot_cells_bokeh(model):

    agent_counts = np.zeros((model.grid.width, model.grid.height))
    w=model.grid.width
    df=grid_values(model)
    df = pd.DataFrame(df.stack(), columns=['value']).reset_index()    
    columns = ['value']
    x = [(i, "@%s" %i) for i in columns]    
    hover = HoverTool(
        tooltips=x, point_policy='follow_mouse')
    colors = Category10[3]
    mapper = LinearColorMapper(palette=colors, low=df.value.min(), high=df.value.max())
    p = figure(plot_width=500,plot_height=500, tools=[hover], x_range=(-1,w), y_range=(-1,w))
    p.rect(x="level_0", y="level_1", width=1, height=1,
       source=df,
       fill_color={'field':'value', 'transform': mapper},
       line_color='black')
    p.background_fill_color = "black"
    p.grid.grid_line_color = None    
    p.axis.axis_line_color = None
    p.toolbar.logo = None
    return p

In [26]:
trans = 0.25 #0.25
death = 0.001 #0.01

In [ ]:
save_folder = 'imag'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [57]:
pn.extension()
plot_pane = pn.pane.Bokeh()
grid_pane = pn.pane.Bokeh()
pn.Row(plot_pane,grid_pane,sizing_mode='stretch_width')

steps=30
pop=900
model = InfectionModel(pop, 30, 30, ptrans=trans, death_rate=death) #0.25, 0.01
for i in range(steps):
    model.step()    
    p1=plot_states_bokeh(model,title='step=%s' %i)
    plot_pane.object = p1
    p2=plot_cells_bokeh(model)
    grid_pane.object = p2
    time.sleep(0.2)
    show(p2)
    name = "./imag/img"+str(i).zfill(3)+".png"
    plt.savefig(name)
    plt.clf()
    

show(p1)
show(p2)
print("Blue = Susceptible, Green = Infected, Orange = Removed\n")
print(get_column_data(model))

Blue = Susceptible, Green = Infected, Orange = Removed

      Susceptible  Infected  Removed
Step                                
0             889        11        0
1             887        13        0
2             882        18        0
3             875        25        0
4             866        34        0
5             862        38        0
6             850        50        0
7             843        56        1
8             828        71        1
9             814        85        1
10            800        99        1
11            783       116        1
12            765       134        1
13            751       145        3
14            729       165        4
15            704       188        6
16            680       211        7
17            650       237       11
18            628       258       11
19            586       297       14
20            549       332       16
21            510       369       18
22            471       397       29
23            439  

<Figure size 432x288 with 0 Axes>

In [58]:
from PIL import Image
import matplotlib.pyplot as plt, glob, os
import IPython.display as IPdisplay

# create a tuple of display durations, one for each frame
first_last = 100 #show the first and last frames for 100 ms
standard_duration = 5 #show all other frames for 5 ms
durations = tuple([first_last] + [standard_duration] * (5*steps-steps - 2) + [first_last])

# load all the static images into a list
images = [Image.open(image) for image in glob.glob('{}/*.png'.format(save_folder))]
gif_filepath = 'imag/animated-epidemic.gif'

# save as an animated gif
gif = images[0]
gif.info['duration'] = durations #ms per frame
gif.info['loop'] = 0 #how many times to loop (0=infinite)
gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])

# verify that the number of frames in the gif equals the number of image files and durations
Image.open(gif_filepath).n_frames == len(images) == len(durations)

False

In [59]:
Image.open(gif_filepath).n_frames

1

In [45]:
len(images), len(durations)

(32, 32)

In [60]:
IPdisplay.Image(url=gif_filepath)

In [14]:
pn.extension()
plot_pane = pn.pane.Bokeh()
grid_pane = pn.pane.Bokeh()
pn.Row(plot_pane,grid_pane,sizing_mode='stretch_width')

steps=81
pop=900
model = InfectionModel(pop, 30, 30, ptrans=trans, death_rate=death)

for i in range(steps):
    model.step()    
    p1 = plot_states_bokeh(model,title='step=%s' %i)
    plot_pane.object = p1
    p2 = plot_cells_bokeh(model)
    grid_pane.object = p2
    time.sleep(0.2)
    
show(p1)
show(p2)
print("Blue = susceptible, Orange = Infected, Green = Removed\n")
print(get_column_data(model))

Blue = susceptible, Orange = Infected, Green = Removed

      Susceptible  Infected  Removed
Step                                
0             887        13        0
1             879        21        0
2             876        24        0
3             870        30        0
4             862        37        0
...           ...       ...      ...
76              9         1      867
77              9         1      867
78              9         1      867
79              9         0      868
80              9         0      868

[81 rows x 3 columns]
